# Лекция 8: Современные подходы в машинном обучении

В этой лекции будут обзорно рассмотрены актуальные темы, которые могут встретиться в литературе или на конференциях.

- Рассмотрим альтернативный подход для использования нейронных сетей - графовые нейросети;
- Обсудим интерпретируемость моделей и инструменты для этого;
- Обучение с подкреплением как самую часто возникающую тему при реализации моделей в проде или проведении симуляций
- AutoML в качестве высокоуровнего подхода к использованию моделей
- Диффузионные модели - основной инструмент для решения задач генерации изображений и видео.

## Графовые нейросети (Graph Neural Networks, GNN)

Графовые нейросети предназначены для обработки данных, представленных в виде графов, где вершины – объекты, а ребра – их взаимосвязи. Такие модели находят применение в:
- социальных сетях, 
- биоинформатике и химии в целом, 
- рекомендационных системах.

### Описание

Рассмотрим граф $ G = (V, E) $, где:
- $ V = \{v_1, v_2, \dots, v_n\} $ — множество вершин (узлов).
- $ E \subseteq V \times V $ — множество рёбер (связей) между узлами.

Каждая вершина $ v_i $ имеет вектор признаков $ \mathbf{x}_i \in \mathbb{R}^d $ (например, текст, атрибуты, числовые показатели - любые идентифицирующие вершину параметры). Все признаки можно собрать в матрицу:
$$
X \in \mathbb{R}^{n \times d}
$$
Связи между вершинами описываются матрицей смежности:
$$
A \in \mathbb{R}^{n \times n}, \quad \text{где } A_{ij} = \begin{cases} 1, & \text{если есть ребро между } v_i \text{ и } v_j \\ 0, & \text{иначе} \end{cases}
$$

Одним из классических вариантов GNN является **GCN** (Graph Convolutional Network), предложенный Kipf & Welling (2016). Основная идея слоя GCN заключается в агрегации признаков соседей и обновлении представления узлов. На каждом шаге все вершины отправляют сообщения своим соседям, после чего обновляют свое состояние в соответствии с полученными новыми сообщениями - Message Passing.

#### Алгоритм работы одного слоя GCN:
1. **Агрегация:** Для каждой вершины собрать признаки её соседей (среднее, максимум, RNN и другие).
2. **Линейное преобразование:** Применить обучаемую матрицу весов.
3. **Нелинейность:** Пропустить результат через функцию активации.

При использовании множества слоев возникает проблема размытия - Over-smoothing. На ребра могут быть добавлены веса.

#### Математическая формулировка
Для слоя $ l $, в случае использования нормализации, обновление признаков записывается так:
$$
H^{(l+1)} = \sigma\left(\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H^{(l)} W^{(l)}\right)
$$
где:
- $ H^{(l)} \in \mathbb{R}^{n \times d_l} $ — матрица признаков на $ l $-ом слое ($ H^{(0)} = X $).
- $\tilde{A} = A + I$ — матрица смежности с добавлением самосвязей (единичная матрица $ I $ добавляется для учета собственного узла).
- $\tilde{D}$ — диагональная матрица, где $ \tilde{D}_{ii} = \sum_j \tilde{A}_{ij} $. Показывает количество связей для каждой вершины.
- $ W^{(l)} $ — матрица весов $ l $-го слоя.
- $\sigma(\cdot)$ — нелинейная функция активации.

**Пояснения:**
- Матрица смежности графа - квадратная матрица из нулей и единиц размера $ n \times n $, где единицы в стоят в позициях свеянных между собой вершин.
- Матрица $\tilde{D}$ делает нормализацию.

Кроме того, есть альтернативный вариант - использование Attention.

**Алгоритм:**
- Для каждой пары (для каждого из соседей определенного ребра) объединяем вектора признаков в один вектор.
- Применяем к нему линейный слой, функцию активации и Softmax.
- полученные значения используются в качестве дополнительных весов при аналогичной предыдущей реализации свертки.

Такой подход позволяет каждому узлу учитывать информацию от соседей, а нормировка через $\tilde{D}^{-1/2}$ помогает избежать числовых нестабильностей при работе с разреженными матрицами смежности.

### Пример

Библиотеки, такие как [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/) и [DGL](https://www.dgl.ai/), позволяют быстро создавать и обучать GNN.

Для практической демонстрации воспользуемся классическим датасетом **Cora**:
- В датасете [Cora](https://paperswithcode.com/dataset/cora) содержится 2708 научных публикаций (узлов), 5429 ссылок между ними (рёбер) и 7 категорий (тематика публикаций).
- Cora широко используется для тестирования методов классификации на графах.

In [2]:
# !pip install torch_geometric -q

In [ ]:
import torch
from torch_geometric.datasets import Planetoid
import networkx as nx
import matplotlib.pyplot as plt

# Загрузка датасета Cora
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Преобразуем данные в граф NetworkX для визуализации.
# Для наглядности берем подграф из первых 100 узлов.
edge_index = data.edge_index.numpy()
G = nx.Graph()
G.add_edges_from(edge_index.T)
sub_nodes = list(G.nodes())[:100]
subG = G.subgraph(sub_nodes)

plt.figure(figsize=(8, 8))
nx.draw(subG, with_labels=True, node_size=300, font_size=8)
plt.title("Подграф датасета Cora (100 узлов)")
plt.show()

In [ ]:
data

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN(dataset.num_features, 16, dataset.num_classes)
print(model)

In [ ]:
import torch.optim as optim
from sklearn.manifold import TSNE

optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    logits, accs = model(data), []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

for epoch in range(200+1):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d} Loss: {loss:.4f} Train: {train_acc:.4f} Val: {val_acc:.4f} Test: {test_acc:.4f}')

model.eval()
with torch.no_grad():
    embeddings = model.conv1(data.x, data.edge_index)
    embeddings = F.relu(embeddings)

embeddings_2d = TSNE(n_components=2).fit_transform(embeddings.cpu().numpy())

plt.figure(figsize=(8, 8))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=data.y.cpu().numpy(), cmap="jet", s=15)
plt.title("t-SNE визуализация встраиваний узлов (GCN)")
plt.show()

## Интерпретируемость моделей (Explainable AI, XAI)

Методы интерпретируемости помогают понять, почему модель принимает те или иные решения. Это особенно важно для критичных областей (медицина, финансы). Популярные подходы включают:
- **SHAP (SHapley Additive exPlanations)**
- **LIME (Local Interpretable Model-agnostic Explanations)**
- **Integrated Gradients**
- **Attention maps**

### Стандартный подход
Например, градиентный спуск на каждой итерации делает какое-то разбиение, пусти нас интересуют все разбиения сделанные по какому-либо определенному признаку. Каждое такое разбиение дает уменьшение функции потерь, их сумма дает суммарный выигрыш по определенному признаку - Gain. Однако такой подсчет может быть неверным.

### SHAP (SHapley Additive exPlanations)

Первоначальная идея: спрячем один из признаков от модели и обучим ее снова - предсказание изменится. Величина изменения - это вклад признака. Делаем так с остальными. Получается очень дорого... 

Поэтому сделаем иначе:

Метод основан на концепции shapley-value из теории кооперативных игр. Каждый признак рассматривается как «игрок», вносящий вклад в итоговое предсказание модели. SHAP распределяет эффект модели на отдельные признаки так, что сумма вкладов равна разнице между прогнозом для данного наблюдения и базовым (средним) прогнозом.

Для модели $ f $ и наблюдения $ x $ предсказание раскладывается так:  
$$
f(x) = \phi_0 + \sum_{i=1}^{M} \phi_i
$$
где:
- $ \phi_0 $ – базовое значение (например, среднее по выборке),
- $ \phi_i $ – вклад $ i $-го признака, вычисляемый по формуле shapley-value:
$$
\phi_i = \sum_{S \subseteq F \setminus \{i\}} \frac{|S|!(|F|-|S|-1)!}{|F|!} \Bigl( f_{S \cup \{i\}}(x_{S \cup \{i\}}) - f_S(x_S) \Bigr)
$$
При этом: 
- $ F $ – множество всех признаков. Мы рассматриваем все возможные подмножества признаков, которые **не содержат** признак $ i $.
- $ S $ – подмножество признаков,
- $ f_S $ – модель, обученная или оцененная только на признаках из $ S $.

Разберем эту формулу:
- Справа: смотрим ошибку алгоритма с признаком и без него (насколько изменится ответ без использования признака).
- Слева: комбинаторный вес, который учитывает количество возможных порядков (перестановок) признаков. Он гарантирует, что вклад каждого признака будет усреднён по всем возможным порядкам их включения в модель.

**Особенности:**
- Гарантирует локальную точность (сумма вкладов точно объясняет разницу в предсказании).
- Обеспечивает консистентность при изменении модели.

#### LIME (Local Interpretable Model-agnostic Explanations)
 
LIME строит интерпретируемую локальную аппроксимацию сложной модели вокруг конкретного наблюдения. Для этого генерируются искусственные примеры путём незначительных изменений исходного входа, после чего на этих данных обучается простая модель (например, линейная регрессия).

**Особенности:**
- Метод «локален»: объяснение действительно только в окрестности конкретного примера.
- Универсален: применим к любым моделям, так как не требует внутреннего устройства.

#### Integrated Gradients

Этот метод применяется для глубоких нейросетей. Он вычисляет вклад каждого признака, интегрируя градиенты вдоль прямой линии от базового (обычно нулевого) входа до фактического входа $ x $.

Для признака $ i $ вклад определяется как:
$$
IG_i(x) = (x_i - x_i') \times \int_{\alpha=0}^{1} \frac{\partial f\bigl(x' + \alpha (x - x')\bigr)}{\partial x_i} d\alpha
$$
где $ x' $ – базовый вход, а $ f $ – модель. Этот метод обладает свойствами чувствительности и инвариантности к реализации.

#### Attention maps

Attention maps используются в моделях с механизмом внимания (например, в трансформерах). Они визуализируют весовые коэффициенты, показывая, на какие части входа модель обращает больше внимания при формировании предсказания.

**Особенности:**
- В NLP: внимание между токенами предложения.
- В компьютерном зрении: внимание к различным областям изображения.

### Пример

В качестве примера выберем датасет **Breast Cancer** из библиотеки scikit-learn, обладающий понятными и интерпретируемыми признаками (например, «mean radius», «mean texture», и т.д.). Для модели воспользуемся классификатором XGBoost.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import xgboost as xgb

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    test_size=0.3, 
                                                    random_state=42)

model = xgb.XGBClassifier(n_estimators=100, 
                          learning_rate=0.1, 
                          importance_type="weight", 
                          random_state=42)
model.fit(X_train, y_train)

In [ ]:
# !pip install shap -q

In [ ]:
import shap

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test)

In [ ]:
importances = model.feature_importances_
features = data.feature_names

plt.figure(figsize=(10, 6))
plt.barh(features, importances)
plt.xlabel("Feature Importance")
plt.title("Важность признаков по XGBoost")
plt.show()

- **SHAP:** Дает детальную локальную интерпретацию для каждого наблюдения, позволяет увидеть, как признаки влияют на конкретное предсказание, и учитывает взаимодействие признаков.  
- **Feature Importance:** Предоставляет глобальную оценку важности признаков, не раскрывая, как именно они влияют на отдельные прогнозы.

## Reinforcement Learning (Обучение с подкреплением)

Обучение с подкреплением (RL) – это подход, при котором агент (модель) обучается взаимодействовать со средой, получая обратную связь в виде вознаграждения. Основная цель – найти такую стратегию (policy), которая максимизирует суммарное вознаграждение. 

RL применяется в играх, робототехнике, управлении и оптимизации процессов.

Лекции: 
- [Воронцов](https://www.youtube.com/watch?v=iEUrX_eEWNY&t=499s&ab_channel=YandexforML)
- [Нейчев](https://www.youtube.com/watch?v=neYEP75m4bo&ab_channel=%D0%9B%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D0%B8%D0%B9%D0%A4%D0%9F%D0%9C%D0%98)

### Терминология
- **Агент:** обучаемая система, принимающая решения.
- **Среда (Environment):** внешний мир, с которым взаимодействует агент.
- **Состояние (State, $s$):** описание среды в конкретный момент времени.
- **Действие (Action, $a$):** выбор агента, влияющий на состояние среды.
- **Вознаграждение (Reward, $r$):** числовая оценка качества выбранного действия.
- **Политика (Policy, $\pi(a|s)$):** стратегия выбора действий на основе текущего состояния.
- **Функция ценности (Value Function):** оценка качества состояния или пары состояние-действие (например, $V(s)$ или $Q(s,a)$).
- **Преимущество (Advantage, $\hat{A}(s,a)$):** разница между фактическим вознаграждением и ожидаемой ценностью, помогает понять, насколько лучше (или хуже) действие по сравнению со средним.

### Основные стадии обучения
1. **Сбор опыта:** Агент взаимодействует со средой, собирая переходы: $(s_t, a_t, r_t, s_{t+1})$.
2. **Оценка политики:** На основе собранных данных вычисляются значения функции ценности и преимущества.
3. **Обновление политики:** Используя методы градиентного спуска, политика корректируется так, чтобы увеличить вероятность выбора действий, приносящих высокое вознаграждение.
4. **Баланс между исследованием и эксплуатацией:** Агент должен исследовать новые действия (exploration) и использовать уже известные (exploitation).

### Описание ключевых алгоритмов

#### Proximal Policy Optimization (PPO)

PPO – один из самых популярных алгоритмов, сочетающий стабильность и эффективность. Основная идея – ограничить изменение политики на каждом шаге, чтобы избежать чрезмерных обновлений.

Для старой политики $\pi_{\theta_{\text{old}}}$ и новой $\pi_{\theta}$ определяется отношение:
$$
r_t(\theta) = \frac{\pi_{\theta}(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}
$$
Объективная функция с использованием механизма обрезки (clipping):
$$
L^{\text{CLIP}}(\theta) = \hat{\mathbb{E}}_t \left[\min\Bigl( r_t(\theta)\,\hat{A}_t,\; \text{clip}\bigl(r_t(\theta),1-\epsilon,1+\epsilon\bigr)\,\hat{A}_t \Bigr) \right]
$$
где:
- $\hat{A}_t$ – оценка преимущества.
- $\epsilon$ – параметр, определяющий допустимое отклонение от старой политики (обычно $0.1 \dots 0.3$).

Этот метод гарантирует, что обновление политики не приведёт к слишком резкому изменению, что делает обучение более стабильным.

#### Direct Preference Optimization (DPO)

DPO – относительно новый подход, часто используемый в контексте обучения с человеческой обратной связью (например, в RLHF). Вместо явного определения функции вознаграждения, DPO использует парные предпочтения, чтобы оптимизировать политику.

**RLHF (Reinforcement Learning from Human Feedback)** – это подход, при котором обучение модели осуществляется с использованием сигналов, предоставляемых человеком. Обычно сначала обучается базовая модель, затем собирается обратная связь (например, предпочтения или оценки) от пользователей, и на основе этой информации дообучается политика (часто с использованием методов, подобных DPO). Такой подход позволяет сделать модель более соответствующей ожиданиям и ценностям конечных пользователей.

Для пары предпочтительных действий $a^+$ (предпочтительный) и $a^-$ (непредпочтительный) в одном и том же состоянии $s$, DPO стремится оптимизировать такую функцию потерь:
$$
L(\theta) = -\mathbb{E}_{(s,a^+,a^-)} \log \frac{\exp\bigl(\beta \log \pi_\theta(a^+|s)\bigr)}{\exp\bigl(\beta \log \pi_\theta(a^+|s)\bigr) + \exp\bigl(\beta \log \pi_\theta(a^-|s)\bigr)}
$$
где:
- $\pi_\theta(a|s)$ – политика с параметрами $\theta$.
- $\beta$ – параметр масштаба, регулирующий чувствительность к разнице между лог-вероятностями.

Таким образом, модель обучается так, чтобы вероятность выбора предпочтительного действия была выше, чем вероятность выбора непредпочтительного.

#### Другие алгоритмы

- **DQN (Deep Q-Network):** Использует нейросеть для аппроксимации Q-функции $Q(s,a)$. Обновление происходит через минимизацию ошибки между целевым и текущим Q-значением.
- **A3C (Asynchronous Advantage Actor-Critic):** Параллельно обучает несколько агентов, что позволяет ускорить сбор опыта и уменьшить корреляцию между наблюдениями.
- **SAC (Soft Actor-Critic):** Алгоритм, оптимизирующий энтропийное регуляризированное вознаграждение, что приводит к более устойчивому исследованию.

Пример с использованием [задачи](https://www.gymlibrary.dev/environments/classic_control/cart_pole/)

In [10]:
# !pip install gym stable_baselines3 shimmy -q

In [ ]:
import gym
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
import numpy as np

env = gym.make('CartPole-v1')

model = PPO("MlpPolicy", env, verbose=1)

total_timesteps = 20000
model.learn(total_timesteps=total_timesteps)

episodes = 20
episode_rewards = []

for episode in range(episodes):
    obs, info = env.reset()
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
    episode_rewards.append(total_reward)

plt.figure(figsize=(10, 5))
plt.plot(range(1, episodes+1), episode_rewards, marker='o')
plt.title("Вознаграждения агента в задаче CartPole")
plt.xlabel("Эпизод")
plt.ylabel("Суммарное вознаграждение")
plt.grid(True)
plt.show()

## AutoML

AutoML (Automated Machine Learning) – это набор методов и инструментов, направленных на автоматизацию всего процесса создания модели машинного обучения. 

Это включает:
- **Предобработку данных:** автоматический выбор и применение методов очистки, нормализации, отбора признаков и создания новых признаков.
- **Выбор модели:** определение наиболее подходящих алгоритмов для конкретной задачи (например, деревья решений, градиентный бустинг, SVM и др.).
- **Тюнинг гиперпараметров:** поиск оптимальных значений гиперпараметров моделей с использованием методов, таких как случайный поиск, байесовская оптимизация, эволюционные алгоритмы и т.д.
- **Сборка пайплайна:** комбинирование этапов предобработки, выбора модели и тюнинга в единую автоматизированную систему.
- **Ансамблирование:** объединение нескольких моделей для повышения качества предсказаний.

Существуют инструменты, такие как [auto-sklearn](https://automl.github.io/auto-sklearn/master/) или [TPOT](http://epistasislab.github.io/tpot/), которые помогают быстро найти оптимальное решение для задачи без глубоких знаний в ML.

### Как работает AutoML?

1. **Определение пространства поиска:**
   AutoML-фреймворки формируют множество возможных решений (пайплайнов) с различными комбинациями алгоритмов, методов предобработки и наборов гиперпараметров.

2. **Поисковые алгоритмы:**
   Для исследования пространства решений применяются:
   - Случайный поиск – простое, но не всегда эффективное.
   - Байесовская оптимизация – использование апостериорного распределения для целевой функции.
   - Эволюционные алгоритмы – имитация естественного отбора для эволюции пайплайнов.
   - Методы на основе градиентного спуска – для некоторых непрерывных параметров.

3. **Кросс-валидация и метрики:**  
   Каждая комбинация оценивается с использованием кросс-валидации, чтобы получить надежную оценку качества модели.

4. **Ансамблирование:**  
   После поиска лучших моделей часто строится ансамбль, в котором взвешиваются отдельные модели для получения более стабильного результата.

### Плюсы и минусы

**Плюсы:**
- Автоматизация: позволяет значительно сократить время и усилия, требуемые для экспериментов с моделями.
- Доступность: пользователи без глубоких знаний в ML могут получить конкурентоспособные модели.
- Эффективность: часто находит нестандартные комбинации предобработки и моделей, которые сложно подобрать вручную.
- Ансамблирование: встроенные методы ансамблирования могут повышать качество предсказаний.

**Минусы:**
- Вычислительные затраты: поиск по большому пространству гиперпараметров может требовать значительных вычислительных ресурсов.
- Черный ящик: конечный пайплайн может быть сложным для интерпретации, что снижает прозрачность.
- Ограничения по кастомизации: автоматизированные решения не всегда могут удовлетворить специфическим требованиям задачи.
- Время обучения: полный поиск может занять много времени, особенно на больших датасетах.


Пример с использованием AutoML для sklearn (linux)

In [ ]:
# !pip install auto-sklearn -q

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import autosklearn.classification

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30
)
automl.fit(X_train, y_train)

print(automl.sprint_statistics())

In [ ]:
cv_results = automl.cv_results_
mean_scores = cv_results['mean_test_score']
pipeline_indices = range(len(mean_scores))

plt.figure(figsize=(10, 6))
plt.bar(pipeline_indices, mean_scores, color='skyblue')
plt.xlabel('Индекс пайплайна')
plt.ylabel('CV Score')
plt.title('Кросс-валидационные оценки для различных пайплайнов')
plt.show()

In [ ]:
models_with_weights = automl.get_models_with_weights()

model_names = [type(model).__name__ for weight, model in models_with_weights]
model_weights = [weight for weight, model in models_with_weights]

plt.figure(figsize=(10, 6))
plt.bar(model_names, model_weights, color='lightgreen')
plt.xlabel("Тип модели")
plt.ylabel("Вес в ансамбле")
plt.title("Состав ансамбля, построенного AutoML")
plt.xticks(rotation=45)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = automl.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность на тестовой выборке:", accuracy)

## Диффузионные модели

Диффузионные модели – это класс генеративных моделей, в основе которых лежит процесс постепенного добавления шума к данным (прямой процесс) и последующее восстановление исходных данных через обратный процесс. Такие модели успешно применяются для генерации изображений, аудио и других типов данных.

### Математическое описание

#### Прямой (forward) процесс

Пусть $ x_0 $ — исходное изображение или данные. Прямой процесс заключается в последовательном добавлении шума на каждом из $ T $ шагов. На шаге $ t $ процесс определяется следующим образом:

$$
q(x_t \mid x_{t-1}) = \mathcal{N}(x_t; \sqrt{1-\beta_t}\, x_{t-1},\, \beta_t \mathbf{I})
$$

где:
- $ \beta_t $ — коэффициент шума на шаге $ t $ (обычно выбирается небольшой, например, в диапазоне $ [10^{-4}, 0.02] $),
- $ \mathbf{I} $ — единичная матрица.

Из-за свойств марковской цепи можно записать распределение на любом шаге $ t $ напрямую от $ x_0 $:

$$
q(x_t \mid x_0) = \mathcal{N}\!\Bigl(x_t; \sqrt{\bar{\alpha}_t}\, x_0,\,(1-\bar{\alpha}_t)\mathbf{I}\Bigr)
$$
где:
$$
\alpha_t = 1 - \beta_t,\quad \bar{\alpha}_t = \prod_{s=1}^{t} \alpha_s.
$$

#### Обратный (reverse) процесс

Цель генеративной модели – восстановить $ x_0 $ из чистого шума $ x_T \sim \mathcal{N}(0,\mathbf{I}) $ посредством обратного процесса. Этот процесс аппроксимируется с помощью нейросети с параметрами $ \theta $:

$$
p_\theta(x_{t-1} \mid x_t) = \mathcal{N}\!\Bigl(x_{t-1};\, \mu_\theta(x_t, t),\, \Sigma_\theta(x_t, t)\Bigr).
$$

Обучение модели часто проводится с использованием упрощённой целевой функции, называемой _noise prediction objective_. Суть её в том, чтобы предсказать добавленный шум $\epsilon$ в ходе прямого процесса:

$$
L_{\text{simple}} = \mathbb{E}_{x_0,\, \epsilon,\, t}\!\Bigl[\bigl\|\epsilon - \epsilon_\theta\bigl(\sqrt{\bar{\alpha}_t}\, x_0 + \sqrt{1-\bar{\alpha}_t}\, \epsilon,\ t\bigr)\bigr\|^2\Bigr],
$$

где:
- $ \epsilon \sim \mathcal{N}(0, \mathbf{I}) $ – случайный шум,
- $ \epsilon_\theta(x_t, t) $ – предсказание модели шума на шаге $ t $.

### Процесс генерации

После обучения генерация нового примера происходит итеративно:
1. Начинаем с $ x_T \sim \mathcal{N}(0,\mathbf{I}) $.
2. Для каждого шага $ t $ от $ T $ до $ 1 $ выбираем:
   $$
   x_{t-1} \sim p_\theta(x_{t-1}\mid x_t).
   $$
3. Итоговый $ x_0 $ – это сгенерированное изображение или данные.

### Архитектура диффузионных моделей

### U-Net

Большинство современных диффузионных моделей используют **U-Net** архитектуру для предсказания шума $\epsilon_\theta(x_t, t)$. Ключевые элементы:
- **Encoder:** Последовательность свёрточных слоёв, уменьшающих размерность, чтобы зафиксировать глобальные особенности.
- **Decoder:** Последовательность свёрточных слоёв, увеличивающих размерность для восстановления исходного изображения.
- **Skip connections:** Пропускные связи между соответствующими слоями encoder и decoder для сохранения деталей.
- **Positional / Temporal Embeddings:** Кодирование номера шага $ t $ для информирования сети о текущей степени зашумлённости.

### Дополнительные архитектурные решения

- **Attention Mechanisms:** Включение слоёв внимания для захвата длиннозависимых зависимостей (например, в Stable Diffusion используется cross-attention для интеграции текстовых подсказок).
- **Variational Autoencoder (VAE):** В некоторых моделях, таких как Stable Diffusion, используется VAE для кодирования изображений в более компактное латентное пространство перед применением диффузионного процесса.

### Построение модели с нуля

Чтобы реализовать диффузионную модель с нуля:
1. **Определить прямой процесс:** Выбрать схему $ \beta_t $ для добавления шума.
2. **Построить U-Net:** Реализовать U-Net, принимающий на вход зашумлённое изображение $ x_t $ и временной индекс $ t $ (с эмбеддингом).
3. **Функция потерь:** Определить loss-функцию $ L_{\text{simple}} $ для обучения модели предсказывать шум.
4. **Обучение:** Обучать сеть на датасете изображений, постепенно уменьшая шум.
5. **Генерация:** Реализовать обратный процесс, начиная с $ x_T \sim \mathcal{N}(0, \mathbf{I}) $ и восстанавливая $ x_0 $.

Практический пример с использованием готовой модели

In [1]:
# !pip install diffusers accelerate -q

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

prompt = "A surreal landscape with vivid colors, intricate details, and a dreamlike atmosphere"
result = pipe(prompt)
image = result.images[0]

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.axis("off")
plt.title("Сгенерированное изображение с помощью Stable Diffusion")
plt.show()

### Процесса работы

- **Энкодер текста:** Входной текст обрабатывается текстовым энкодером (например, CLIP) для получения условного представления.
- **Латентное пространство:** Изображение кодируется в компактное латентное пространство с помощью VAE.
- **Обратный диффузионный процесс:** U-Net принимает на вход зашумлённое латентное представление и временной индекс $ t $ (с дополнительными эмбеддингами) и предсказывает шум, постепенно приближая латентное представление к "чистому" изображению.
- **Декодирование:** После обратного процесса латентное представление преобразуется в изображение через VAE-декодер.
- **Архитектура:** Модель включает U-Net с вниманием, слои нормализации и позиционные эмбеддинги, что позволяет точно управлять процессом восстановления изображения.